# Load Data

In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
dataset_dir = '../ExtraSensory.per_uuid_features_labels'
label_dir = '../ExtraSensory.per_uuid_original_labels'
data = pd.DataFrame()
for filename in tqdm(os.listdir(dataset_dir)):
    key = filename[:filename.find('.')]
    csv = pd.read_csv(os.path.join(dataset_dir, filename), index_col=0)
    label = pd.read_csv(os.path.join(label_dir, key+'.original_labels.csv.gz'), index_col=0)
    csv = csv.join(label.drop('label_source', axis=1))
    csv = pd.concat([csv], keys=[key], names=['filename'])
    data = csv if data.empty else data.append(csv)

In [27]:
data.columns = pd.MultiIndex.from_tuples([tuple(i.split(':')) for i in data.columns])

In [10]:
data.shape

(377346, 393)

In [11]:
data.head()

raw_acc:magnitude_stats:mean  \
filename                             timestamp                                  
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                      1.004012   
                                     1440000515                      1.002203   
                                     1440000575                      1.002147   
                                     1440000635                      1.001937   
                                     1440000695                      1.005683   

                                                 raw_acc:magnitude_stats:std  \
filename                             timestamp                                 
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                     0.040490   
                                     1440000515                     0.011955   
                                     1440000575                     0.005930   
                                     1440000635                     0.005161   
                                     1440000695                     0.066262   

                                                 raw_acc:magnitude_stats:moment3  \
filename                             timestamp                                     
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                         0.100981   
                                     1440000515                        -0.004731   
                                     1440000575                         0.006193   
                                     1440000635                        -0.007264   
                                     1440000695                         0.166331   

                                                 raw_acc:magnitude_stats:moment4  \
filename                             timestamp                                     
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                         0.178834   
                                     1440000515                         0.025712   
                                     1440000575                         0.014327   
                                     1440000635                         0.015998   
                                     1440000695                         0.291310   

                                                 raw_acc:magnitude_stats:percentile25  \
filename                             timestamp                                          
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                              1.000016   
                                     1440000515                              0.999446   
                                     1440000575                              0.999737   
                                     1440000635                              0.999730   
                                     1440000695                              0.999031   

                                                 raw_acc:magnitude_stats:percentile50  \
filename                             timestamp                                          
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                              1.003875   
                                     1440000515                              1.002202   
                                     1440000575                              1.002325   
                                     1440000635                              1.001946   
                                     1440000695                              1.003126   

                                                 raw_acc:magnitude_stats:percentile75  \
filename                             timestamp                                          
11B5EC4D-4133-4289-B475-4E737182A406 1440000455                              1.006938   
                                     1440000515                              1.004927   
                                     1440000575                              1.004686   
                                     1440000635  

# Train/test split

In [3]:
labels_col = [i for i in data.columns if i.startswith('original_label')]
main_labels_col = labels_col[:7]

['original_label:LYING_DOWN',
 'original_label:SITTING',
 'original_label:STANDING_IN_PLACE',
 'original_label:STANDING_AND_MOVING',
 'original_label:WALKING',
 'original_label:RUNNING',
 'original_label:BICYCLING',
 'original_label:LIFTING_WEIGHTS',
 'original_label:PLAYING_BASEBALL',
 'original_label:PLAYING_BASKETBALL',
 'original_label:PLAYING_LACROSSE',
 'original_label:SKATEBOARDING',
 'original_label:PLAYING_SOCCER',
 'original_label:PLAYING_FRISBEE',
 'original_label:EXERCISING',
 'original_label:STRETCHING',
 'original_label:YOGA',
 'original_label:ELLIPTICAL_MACHINE',
 'original_label:TREADMILL',
 'original_label:STATIONARY_BIKE',
 'original_label:COOKING',
 'original_label:CLEANING',
 'original_label:GARDENING',
 'original_label:DOING_LAUNDRY',
 'original_label:MOWING_THE_LAWN',
 'original_label:RAKING_LEAVES',
 'original_label:VACUUMING',
 'original_label:WASHING_DISHES',
 'original_label:WASHING_CAR',
 'original_label:MANUAL_LABOR',
 'original_label:DANCING',
 'original_la